#**Importando Bibliotecas**

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import numpy as np
import pandas as pd


#**Carregar dados**

In [ ]:
! wget --no-check-certificate --content-disposition 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.21082/dados?formato=csv'

URL transformed to HTTPS due to an HSTS policy
--2021-09-25 21:17:08--  https://api.bcb.gov.br/dados/serie/bcdata.sgs.21082/dados?formato=csv
Resolving api.bcb.gov.br (api.bcb.gov.br)... 200.218.208.96
Connecting to api.bcb.gov.br (api.bcb.gov.br)|200.218.208.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2641 (2.6K) [text/csv]
Saving to: ‘dados?formato=csv’

dados?formato=csv   100%[===================>]   2.58K  --.-KB/s    in 0s      

2021-09-25 21:17:09 (343 MB/s) - ‘dados?formato=csv’ saved [2641/2641]



In [ ]:
! wget --no-check-certificate --content-disposition 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.4390/dados?formato=csv'

URL transformed to HTTPS due to an HSTS policy
--2021-09-25 21:17:12--  https://api.bcb.gov.br/dados/serie/bcdata.sgs.4390/dados?formato=csv
Resolving api.bcb.gov.br (api.bcb.gov.br)... 200.218.208.96
Connecting to api.bcb.gov.br (api.bcb.gov.br)|200.218.208.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8999 (8.8K) [text/csv]
Saving to: ‘bcdata.sgs.4390.csv.1’

bcdata.sgs.4390.csv 100%[===================>]   8.79K  --.-KB/s    in 0.001s  

2021-09-25 21:17:13 (12.7 MB/s) - ‘bcdata.sgs.4390.csv.1’ saved [8999/8999]



In [ ]:
## Devido ao tamanho da base 'Operações do tesouro direto' e a fim de  agilizar o processo de importação, essa foi carregada manualmente via google.drive. Caso necessário ultilizar webscraping, segue abaixo o código

##! wget --no-check-certificate --content-disposition 'https://www.tesourotransparente.gov.br/ckan/dataset/78739a33-4d2f-4e35-88fd-65f1ccbe81c4/resource/4100d614-d1ad-4b62-9435-84f7943e46f3/download/OperacoesTesouroDireto.csv'

In [ ]:
db_inadimplencia = pd.read_csv('bcdata.sgs.21082.csv',encoding='UTF-8',sep= ';')

In [ ]:
db_selic_acumulada = pd.read_csv('dados?formato=csv',encoding='UTF-8',sep= ';')

In [ ]:
db_tesouro_direto = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Inteligência de negócios/OperacoesTesouroDireto.csv',encoding='UTF-8',sep= ';')

##**Tratando os campos de data**

In [ ]:
db_inadimplencia.data=pd.to_datetime(db_inadimplencia.data, format= '%d/%m/%Y')


In [ ]:
db_selic_acumulada.data=pd.to_datetime(db_selic_acumulada.data, format= '%d/%m/%Y')

In [ ]:

db_tesouro_direto['Data da Operacao']=pd.to_datetime(db_tesouro_direto['Data da Operacao'], format= '%d/%m/%Y')


##**Autenticar projeto**

In [ ]:
auth.authenticate_user()
project_id = 'maua-inteligencia-de-negocios'
client = bigquery.Client(project=project_id)

##**Adicionar dados no BQ**

In [ ]:
dataset_ref = bigquery.Dataset(project_id+'.dados_bacen')
dataset_ref = client.create_dataset(dataset_ref)

In [ ]:
table_ref = dataset_ref.table("TB_inadimplencia")

In [ ]:
job = client.load_table_from_dataframe(db_inadimplencia, table_ref, 
                                       job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job.result() 

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'valor'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


In [ ]:
table_ref = dataset_ref.table("TB_selic_acumulada")

In [ ]:
job = client.load_table_from_dataframe(db_selic_acumulada, table_ref, 
                                       job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job.result() 

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'valor'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


In [ ]:
table_ref = dataset_ref.table("TB_tesouro_direto")

In [ ]:
job = client.load_table_from_dataframe(db_tesouro_direto, table_ref, 
                                       job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job.result() 

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'Tipo Titulo'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


##**Montanddo uma tabela 'resumo' para ser linkada ao dashboard**

In [ ]:
### script em SQL

sql = """
WITH 

tesouro as (
SELECT distinct Codigo_do_Investidor,
replace(Quantidade,",",".") as Titulos, 
replace(Valor_da_Operacao,",",".") as Valor_da_Operacao, 
cast (date_trunc(Data_da_Operacao, month) as date) as data_tesouro

FROM `maua-inteligencia-de-negocios.dados_bacen.TB_tesouro_direto` 

),

inadimplencia as (
SELECT cast (date_trunc(data, month) as date) as data_inadimp, 
replace(valor,",",".") as valor_inadimpl_mes, 
FROM `maua-inteligencia-de-negocios.dados_bacen.TB_inadimplencia`
),

selic as (
SELECT cast (date_trunc(data, month) as date) as data_selic, 
replace(valor,",",".") as valor_selic_mes, 
FROM `maua-inteligencia-de-negocios.dados_bacen.TB_selic_acumulada`
)



select count(Codigo_do_Investidor) as qtd_investidores
       ,sum(cast(Titulos as float64)) as qtd_titulos
       ,sum(cast(Valor_da_Operacao as float64)) as Valor_da_Operacao
       ,avg(cast(valor_inadimpl_mes as float64)) as valor_inadimpl_mes
	     ,avg(cast(valor_selic_mes as float64)) as valor_selic_mes
       ,A.data_tesouro


from tesouro as A LEFT JOIN inadimplencia AS B ON   A.data_tesouro = B.data_inadimp        
                  LEFT JOIN selic         AS C ON   A.data_tesouro = C.data_selic
                  
group by data_tesouro
order by data_tesouro desc

"""



In [ ]:
df_resumo = client.query(sql).to_dataframe()

In [ ]:
table_ref = dataset_ref.table("TB_resumo")

In [ ]:
job = client.load_table_from_dataframe(df_resumo, table_ref, 
                                       job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job.result() 

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'data_tesouro'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))
